In [ ]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py

# to plot figures
%matplotlib inline

import pandas as pd

from datetime import datetime

from utils.data_cube_utilities.dc_mosaic import create_hdmedians_multiple_band_mosaic

from swiss_utils.data_cube_utilities.sdc_utilities import create_slc_clean_mask
from swiss_utils.data_cube_utilities.sdc_advutils import composite_fig

import datacube
dc = datacube.Datacube()

The next cell contains the dataset configuration information:
- platform and product (**for this demo SENTINEL 1 data are required**)
- geographical extent
- time period
- measurements (**for this demo VV, VH and VVVH measurements are required**)

You can generate it in three ways:
1. manually from scratch,
2. by manually copy/pasting the final cell content of the [config_tool_indexed](config_tool_indexed.ipynb) notebook,
3. by loading the final cell content of the [config_tool_indexed](config_tool_indexed.ipynb) notebook using the magic `# %load config_cell.txt`.

**Once the %load ran, you need to re-run the cell to set the configuration variables.**

In [ ]:
%load config_cell.txt

In [ ]:
# Let's load and clean a demo dasatet

dataset_in = dc.load(platform = platform, product = product,
                         time = (start_date, end_date),
                         lon = (min_lon, max_lon),
                         lat = (min_lat, max_lat),
                         measurements = measurements,
                         output_crs = 'epsg:27700', resolution = (-10,10))
# the two last arguments are not mandatory as they are included in the product description
# but it's nice to keep them to keep an eye on data representation

# define nodata
dataset_in = dataset_in.where(dataset_in != -9999)

# When using epsg other than 4326, latitude and longitude are renames y and x.
# Let's correct that then the notebook run more smoothly
dataset_in = dataset_in.rename({'x': 'longitude', 'y': 'latitude'})

In [ ]:
# Display the first scene as a RGB composite

composite_fig(dataset_in.isel(time = 0),
              bands = ['VV', 'VH', 'VVVH'],
              hist_str = 'contr',
              title = f"VV,VH,VVVH composite {pd.to_datetime(str(dataset_in.isel(time=0).time.values)).strftime('%Y.%m.%d')}",
              max_size = 16)